# Easy Money - Modelo de propensión a compra

Mensaje de Carol: Una vez que hayamos hecho el análisis de nuestra BBDD nos gustaría poder vender más y mejor nuestros productos, sé que los data scientist tienen una “bola de cristal” y saben qué clientes son más propensos a comprar nuestros productos así que te pido por favor hagas un modelo de propensión a compra, se que tenemos muchos productos, así que te dejo a tu criterio científico qué productos elegir, o pueden ser varios, ya me sorprenderás

Vamos a desarrollar un modelo capaz de **predecir si un cliente es propenso a contratar un producto de Easy Money, teniendo en cuenta sus características**.

## Importación de librerías

In [1]:
# Importamos las librerias necesarias.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder

## Importación de datos

De las tablas que tenemos disponibles en BBDD, las que serán de utilidad para este modelo son customer_sociodemographics, customer_commercial_activity y sales. 
<p>Guardamos los datos de las tablas en dataframes distintos que limpiaremos, y posteriormente uniremos en un único dataframe 


In [2]:
df_cs = pd.read_csv('/Users/alinaoganesyan/Desktop/Master Data Science/Entregables/Proyecto final/Recursos  Proyecto final-20240502/datasets_TFM + diccionario/customer_sociodemographics.csv', index_col=0)
df_cca = pd.read_csv('/Users/alinaoganesyan/Desktop/Master Data Science/Entregables/Proyecto final/Recursos  Proyecto final-20240502/datasets_TFM + diccionario/customer_commercial_activity.csv', index_col=0)
df_sales = pd.read_csv('/Users/alinaoganesyan/Desktop/Master Data Science/Entregables/Proyecto final/Recursos  Proyecto final-20240502/datasets_TFM + diccionario/sales.csv', index_col=0)

## 1: Data Understanding

### 1.1: Análisis Univariante de datos de las distintos datasets que vamos a usar

#### 1.1.1 Analizamos la tabla customer_sociodemographics

In [3]:
# Datos generales 

df_cs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5962924 entries, 0 to 13647308
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   pk_cid        int64  
 1   pk_partition  object 
 2   country_id    object 
 3   region_code   float64
 4   gender        object 
 5   age           int64  
 6   deceased      object 
 7   salary        float64
dtypes: float64(2), int64(2), object(4)
memory usage: 409.4+ MB


In [4]:
# Visualización de los datos
df_cs.head()

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01,ES,29.0,H,35,N,87218.10
1,1050611,2018-01,ES,13.0,V,23,N,35548.74
2,1050612,2018-01,ES,13.0,V,23,N,122179.11
3,1050613,2018-01,ES,50.0,H,22,N,119775.54
4,1050614,2018-01,ES,50.0,V,23,N,NaN


In [5]:
# Recuento de nulos 

df_cs.isna().sum()

pk_cid                0
pk_partition          0
country_id            0
region_code        2264
gender               25
age                   0
deceased              0
salary          1541104
dtype: int64

De entrada, analizamos la columna "deceased" para eliminar los clientes fallecidos y quedarnos con aquellos que actualmente son reales

In [6]:
df_cs['deceased'].value_counts()

deceased
N    5961849
S       1075
Name: count, dtype: int64

In [7]:
df_cs = df_cs[(df_cs['deceased']  == 'N')]
df_cs['deceased'].value_counts()

deceased
N    5961849
Name: count, dtype: int64

Ahora que nos hemos quedado solamente con los clientes no fallecidos, esta columna no es de mucha utilidad ya que siempre tiene el mismo valor. La podemos eliminar.

In [8]:
del(df_cs['deceased'])

Analizamos también la columna country_id para ver si vale la pena quedarnos con todos los países de la BBDD 

In [9]:
df_cs['country_id'].value_counts()

country_id
ES    5959602
GB        441
FR        225
DE        199
US        195
CH        189
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
LU         17
ET         17
QA         17
CI         17
SA         17
CM         17
SN         17
MR         17
NO         17
RU         17
CO         17
GA         17
GT         17
DO         17
SE         16
DJ         11
PT         11
JM         11
RO          9
HU          8
DZ          7
PE          4
Name: count, dtype: int64

Vemos que los valores de los países distintos a ES son muy pequeños y no aportarán demasiado valor a nuestro modelo. Nos quedamos solamente con los registros que son country_id = ES

In [10]:
df_cs = df_cs[(df_cs['country_id']  == 'ES')]
df_cs['country_id'].value_counts()

country_id
ES    5959602
Name: count, dtype: int64

Ahora que nos hemos quedado solamente con los clientes de ES, esta columna no es de mucha utilidad ya que siempre tiene el mismo valor. La podemos eliminar.

In [11]:
del(df_cs['country_id'])

Según el diccionario de las tablas, region_code es el código de provincia. Este dato sí puede ser relevante, pero vemos que tiene nulos. Como estamos ante un modelo de clasificación binaria, podemos rellenar los nulos con un valor como -1, para que se sean agrupados en un bloque propio.

In [12]:
df_cs['region_code'].fillna(-1, inplace=True)

/var/folders/pb/jt3p2kyx2hx__2r9dsl0gsv80000gn/T/ipykernel_98262/1372854579.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cs['region_code'].fillna(-1, inplace=True)


La columna gender también tenía nulos, pero son muy pocos en relación al total del dataset. Analizamos cuántos nulos quedan después de los cambios ya realizados y elimnamos estas filas por ser muy pocas

In [13]:
df_cs['gender'].isna().sum()

25

In [14]:
df_cs = df_cs.dropna(subset =['gender'])

Analizamos la columna salary 

In [15]:
df_cs['salary'].value_counts()

salary
451931.22    709
288997.44    546
135522.15    466
128318.52    458
104563.80    436
            ... 
127111.23      1
100303.05      1
125135.46      1
188185.68      1
167276.64      1
Name: count, Length: 257626, dtype: int64

In [16]:
df_cs['salary'].describe()

count    4.420911e+06
mean     1.155810e+05
std      2.000189e+05
min      1.202730e+03
25%      6.141354e+04
50%      8.843742e+04
75%      1.313081e+05
max      2.889440e+07
Name: salary, dtype: float64

Vemos que todos los valores son positivos, que la diferencia entre el salario mayor y el salario menor es muy grande, y que la media por tanto es muy elevada también. Igualmente, según el diccionario de la BBDD, se trata del salario de la unidad familiar y consideramos que este valor es importante a la hora de determinar la contratación. Al haber muchos nulos, y al ser el modelo de clasificación binaria, vamos a asignar a los nulos el valor -1

In [17]:
df_cs['salary'].fillna(-1, inplace=True)

/var/folders/pb/jt3p2kyx2hx__2r9dsl0gsv80000gn/T/ipykernel_98262/3059662271.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cs['salary'].fillna(-1, inplace=True)


Transformamos la columna region_code de float a int

In [18]:
df_cs["region_code"] = df_cs["region_code"].astype(int)

Transformamos la columna pk_partition a fecha

In [19]:
df_cs["pk_partition"] = pd.to_datetime(df_cs["pk_partition"])

Finalmente, la tabla de customer_sociodemographics quedará así:

In [20]:
df_cs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5959577 entries, 0 to 13647308
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   pk_cid        int64         
 1   pk_partition  datetime64[ns]
 2   region_code   int64         
 3   gender        object        
 4   age           int64         
 5   salary        float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 318.3+ MB


#### 1.1.2 Analizamos la tabla customer_commercial_activity

In [21]:
# Datos generales 

df_cca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5962924 entries, 0 to 13647308
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   pk_cid           int64  
 1   pk_partition     object 
 2   entry_date       object 
 3   entry_channel    object 
 4   active_customer  float64
 5   segment          object 
dtypes: float64(1), int64(1), object(4)
memory usage: 318.5+ MB


In [22]:
# Visualización de los datos
df_cca.head()

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES
1,1050611,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
2,1050612,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
3,1050613,2018-01,2015-08,KHD,0.0,03 - UNIVERSITARIO
4,1050614,2018-01,2015-08,KHE,1.0,03 - UNIVERSITARIO


In [23]:
# Recuento de nulos 

df_cca.isna().sum()

pk_cid                  0
pk_partition            0
entry_date              0
entry_channel      133033
active_customer         0
segment            133944
dtype: int64

In [24]:
pd.set_option('display.max_rows', 100)

In [25]:
df_cca['entry_channel'].value_counts(normalize = True)*100

entry_channel
KHE    53.413469
KFC    15.276786
KHQ    10.125061
KAT     7.137080
KHK     3.948564
KHM     3.029062
KHN     1.859966
KFA     1.355428
KHD     1.301894
RED     1.039488
KHL     0.715897
KHF     0.354329
KHO     0.141461
KAZ     0.096657
KHC     0.089899
KBG     0.028508
KEH     0.012487
KHP     0.011853
007     0.010515
013     0.006810
KAG     0.005746
KAS     0.005232
KBZ     0.003396
KAA     0.003311
KAR     0.002693
KAY     0.002556
KAF     0.002110
KAB     0.001630
KFD     0.001561
KCC     0.001527
KAW     0.001115
KAQ     0.000892
KDH     0.000875
KAE     0.000823
KCH     0.000823
KCB     0.000806
KAD     0.000772
KAM     0.000738
KAH     0.000686
KAK     0.000583
KAJ     0.000566
KDT     0.000395
KFF     0.000377
KBO     0.000360
KEY     0.000343
KGC     0.000292
004     0.000292
KFS     0.000292
KCL     0.000292
KBE     0.000292
KBH     0.000292
KFK     0.000292
KGN     0.000292
KCI     0.000292
KGX     0.000292
KDR     0.000274
KFL     0.000223
KBW     0.000206


Vemos muchos valores de Entry Channel que son muy pocos en comparación con los valores mayoritarios. Elegimos los canales de entrada que tengan una proporción mayor al 1% y agrupamos el resto en Others

In [26]:
value_counts = (df_cca['entry_channel'].value_counts(normalize = True, dropna=True)*100)
df_cca['entry_channel'] = df_cca['entry_channel'].apply(lambda x: x if pd.isna(x) or value_counts.get(x, 0) >= 1 else 'Others')

In [27]:
df_cca['entry_channel'].value_counts(normalize = True)*100

entry_channel
KHE       53.413469
KFC       15.276786
KHQ       10.125061
KAT        7.137080
KHK        3.948564
KHM        3.029062
KHN        1.859966
Others     1.513202
KFA        1.355428
KHD        1.301894
RED        1.039488
Name: proportion, dtype: float64

Analizamos la columna segment para ver los valores únicos, sabiendo que también tiene nulos

In [28]:
df_cca['segment'].value_counts()

segment
03 - UNIVERSITARIO    3900166
02 - PARTICULARES     1830875
01 - TOP                97939
Name: count, dtype: int64

Rellenamos NaN de channel y segment con un valor identificativo como 'Not defined'

In [29]:
df_cca["entry_channel"].fillna('Not defined', inplace=True)
df_cca["segment"].fillna('Not defined', inplace=True)

/var/folders/pb/jt3p2kyx2hx__2r9dsl0gsv80000gn/T/ipykernel_98262/2969524961.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cca["entry_channel"].fillna('Not defined', inplace=True)
/var/folders/pb/jt3p2kyx2hx__2r9dsl0gsv80000gn/T/ipykernel_98262/2969524961.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

In [30]:
df_cca.isna().sum()

pk_cid             0
pk_partition       0
entry_date         0
entry_channel      0
active_customer    0
segment            0
dtype: int64

In [31]:
df_cca['active_customer'].value_counts()

active_customer
0.0    3557573
1.0    2405351
Name: count, dtype: int64

Transformarmos active_customer de float a int

In [32]:
df_cca["active_customer"] = df_cca["active_customer"].astype(int)

Transformamos las columna pk_partition y entry_date a fecha

In [33]:
df_cca["pk_partition"] = pd.to_datetime(df_cca["pk_partition"])
df_cca["entry_date"] = pd.to_datetime(df_cca["entry_date"])

Eliminamos la columna de entry_date ya que la fecha que nos interesa realmente es la fecha de venta dela tabla sales

In [34]:
del(df_cca['entry_date'])

Finalmente, la tabla de customer_commercial_activity quedará así:

In [35]:
df_cca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5962924 entries, 0 to 13647308
Data columns (total 5 columns):
 #   Column           Dtype         
---  ------           -----         
 0   pk_cid           int64         
 1   pk_partition     datetime64[ns]
 2   entry_channel    object        
 3   active_customer  int64         
 4   segment          object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 273.0+ MB


#### 1.1.3 Analizamos la tabla sales

In [36]:
# Datos generales 

df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240773 entries, 0 to 240772
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   pk_sale     240773 non-null  int64  
 1   cid         240773 non-null  int64  
 2   month_sale  240773 non-null  object 
 3   product_ID  240773 non-null  int64  
 4   net_margin  240773 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 11.0+ MB


In [37]:
# Visualización de los datos
df_sales.head()

,pk_sale,cid,month_sale,product_ID,net_margin
0,6666,33620,2018-05-01,2335,952.9
1,6667,35063,2018-06-01,2335,1625.2
2,6668,37299,2018-02-01,2335,1279.7
3,6669,39997,2018-02-01,2335,1511.9
4,6670,44012,2018-02-01,2335,1680.3


In [38]:
# Recuento de nulos 

df_sales.isna().sum()

pk_sale       0
cid           0
month_sale    0
product_ID    0
net_margin    0
dtype: int64

La columna pk_sale es el identificador de la venta. Esta columna no va a ser necesaria para el modelo, por lo que la eliminamos.

In [39]:
del(df_sales['pk_sale'])

La clolumna net_margin nos puede ser útil para decidir qué producto elegir para la predicción de nuestro modelo, por lo que de momento la dejamos.

Transformamos la columna month_sale a fecha

In [40]:
df_sales["month_sale"] = pd.to_datetime(df_sales["month_sale"])

Finalmente, la tabla de sales quedará así:

In [71]:
df_sales.loc[(df_sales['cid'] == 1050659)]

,cid,month_sale,product_ID,net_margin
5496,1050659,2019-01-01,2673,1087.2


In [41]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240773 entries, 0 to 240772
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   cid         240773 non-null  int64         
 1   month_sale  240773 non-null  datetime64[ns]
 2   product_ID  240773 non-null  int64         
 3   net_margin  240773 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 9.2 MB


## 2: Construcción del dataset para el modelo

Vamos a hacer un inner merge de todos los datasets, ya que nos interesa agrupar toda la información analizada en un único dataframe, que contenga toda la información por el id de cliente.

In [44]:
df_merged = pd.merge(
    left = df_cca, 
    right = df_cs,
    on = ('pk_cid', 'pk_partition'),
    how = 'inner')

In [ ]:
df_cca.loc[df_cca['pk_cid'] == 1375586]




,pk_cid,pk_partition,entry_channel,active_customer,segment
0,1375586,2018-01-01,Others,1,02 - PARTICULARES
1047196,1375586,2018-02-01,Others,1,02 - PARTICULARES
1681276,1375586,2018-03-01,Others,1,02 - PARTICULARES
2299297,1375586,2018-04-01,Others,1,02 - PARTICULARES
2714121,1375586,2018-05-01,Others,1,02 - PARTICULARES
3343336,1375586,2018-06-01,Others,1,02 - PARTICULARES
4581564,1375586,2018-07-01,Others,1,02 - PARTICULARES
5075380,1375586,2018-08-01,Others,0,02 - PARTICULARES
6258167,1375586,2018-09-01,Others,0,02 - PARTICULARES
6572487,1375586,2018-10-01,Others,0,02 - PARTICULARES


In [ ]:
df_cs.loc[df_cs['pk_cfid'] == 1375586]

,pk_cid,pk_partition,region_code,gender,age,salary
0,1375586,2018-01-01,29,H,35,87218.1
1047196,1375586,2018-02-01,29,H,35,87218.1
1681276,1375586,2018-03-01,29,H,35,87218.1
2299297,1375586,2018-04-01,29,H,35,87218.1
2714121,1375586,2018-05-01,29,H,35,87218.1
3343336,1375586,2018-06-01,29,H,35,87218.1
4581564,1375586,2018-07-01,29,H,35,87218.1
5075380,1375586,2018-08-01,29,H,36,87218.1
6258167,1375586,2018-09-01,29,H,36,87218.1
6572487,1375586,2018-10-01,29,H,36,87218.1


In [ ]:
df_sales.loc[df_sales['cid'] == 1375586]

,cid,month_sale,product_ID,net_margin
105301,1375586,2018-05-01,3819,56.9


In [46]:
df_merged.loc[df_merged['pk_cid'] == 1375586]

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary
0,1375586,2018-01-01,Others,1,02 - PARTICULARES,29,H,35,87218.1
346132,1375586,2018-02-01,Others,1,02 - PARTICULARES,29,H,35,87218.1
592057,1375586,2018-03-01,Others,1,02 - PARTICULARES,29,H,35,87218.1
830614,1375586,2018-04-01,Others,1,02 - PARTICULARES,29,H,35,87218.1
1107780,1375586,2018-05-01,Others,1,02 - PARTICULARES,29,H,35,87218.1
1356458,1375586,2018-06-01,Others,1,02 - PARTICULARES,29,H,35,87218.1
1790284,1375586,2018-07-01,Others,1,02 - PARTICULARES,29,H,35,87218.1
1987751,1375586,2018-08-01,Others,0,02 - PARTICULARES,29,H,36,87218.1
2486077,1375586,2018-09-01,Others,0,02 - PARTICULARES,29,H,36,87218.1
2752361,1375586,2018-10-01,Others,0,02 - PARTICULARES,29,H,36,87218.1


In [48]:
df_merged.head()

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary
0,1375586,2018-01-01,Others,1,02 - PARTICULARES,29,H,35,87218.10
1,1050611,2018-01-01,KHE,0,03 - UNIVERSITARIO,13,V,23,35548.74
2,1050612,2018-01-01,KHE,0,03 - UNIVERSITARIO,13,V,23,122179.11
3,1050613,2018-01-01,KHD,0,03 - UNIVERSITARIO,50,H,22,119775.54
4,1050614,2018-01-01,KHE,1,03 - UNIVERSITARIO,50,V,23,-1.00


In [49]:
df_merged = pd.merge(
    left = df_merged, 
    right = df_sales,
    left_on = 'pk_cid',
    right_on  = 'cid',
    how = 'inner')

In [ ]:
pd.set_option('display.max_rows', 1000)

In [50]:
df_merged.head(50)

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,cid,month_sale,product_ID,net_margin
0,1375586,2018-01-01,Others,1,02 - PARTICULARES,29,H,35,87218.10,1375586,2018-05-01,3819,56.9
1,1050623,2018-01-01,KHE,1,03 - UNIVERSITARIO,49,H,23,113538.81,1050623,2018-02-01,3819,54.4
2,1050602,2018-01-01,KHE,1,03 - UNIVERSITARIO,37,V,23,91282.71,1050602,2019-04-01,3819,65.1
3,1050580,2018-01-01,KHE,1,03 - UNIVERSITARIO,27,H,23,77320.92,1050580,2018-10-01,3819,53.5
4,1050676,2018-01-01,KFC,0,02 - PARTICULARES,45,V,31,90431.85,1050676,2019-02-01,3819,60.4
5,1050660,2018-01-01,KFC,1,03 - UNIVERSITARIO,41,H,32,94683.27,1050660,2018-05-01,2234,74.2
6,1050660,2018-01-01,KFC,1,03 - UNIVERSITARIO,41,H,32,94683.27,1050660,2018-05-01,1364,5339.1
7,1050647,2018-01-01,KAT,1,02 - PARTICULARES,3,H,38,28359.36,1050647,2019-04-01,2234,88.2
8,1050647,2018-01-01,KAT,1,02 - PARTICULARES,3,H,38,28359.36,1050647,2019-04-01,1364,2426.7
9,1050647,2018-01-01,KAT,1,02 - PARTICULARES,3,H,38,28359.36,1050647,2018-07-01,3819,63.8


In [51]:
df_merged.loc[df_merged['pk_cid'] == 1375586]

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,cid,month_sale,product_ID,net_margin
0,1375586,2018-01-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
108779,1375586,2018-02-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
195146,1375586,2018-03-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
272261,1375586,2018-04-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
363985,1375586,2018-05-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
459543,1375586,2018-06-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
627920,1375586,2018-07-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
753249,1375586,2018-08-01,Others,0,02 - PARTICULARES,29,H,36,87218.1,1375586,2018-05-01,3819,56.9
919677,1375586,2018-09-01,Others,0,02 - PARTICULARES,29,H,36,87218.1,1375586,2018-05-01,3819,56.9
1046964,1375586,2018-10-01,Others,0,02 - PARTICULARES,29,H,36,87218.1,1375586,2018-05-01,3819,56.9


In [52]:
df_merged.loc[df_merged['pk_cid'] == 1375586]

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,cid,month_sale,product_ID,net_margin
0,1375586,2018-01-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
108779,1375586,2018-02-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
195146,1375586,2018-03-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
272261,1375586,2018-04-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
363985,1375586,2018-05-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
459543,1375586,2018-06-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
627920,1375586,2018-07-01,Others,1,02 - PARTICULARES,29,H,35,87218.1,1375586,2018-05-01,3819,56.9
753249,1375586,2018-08-01,Others,0,02 - PARTICULARES,29,H,36,87218.1,1375586,2018-05-01,3819,56.9
919677,1375586,2018-09-01,Others,0,02 - PARTICULARES,29,H,36,87218.1,1375586,2018-05-01,3819,56.9
1046964,1375586,2018-10-01,Others,0,02 - PARTICULARES,29,H,36,87218.1,1375586,2018-05-01,3819,56.9


In [58]:
df_con_reg = pd.read_csv('/Users/alinaoganesyan/Desktop/Master Data Science/Entregables/Proyecto final/git-proyecto/data/reg_pension_plan.csv', sep = ';')

In [64]:
df_con_reg.head()

,pk_cid,pk_partition,reg_pension_plan
0,15891,2018-01-07,0
1,15891,2018-01-08,0
2,16063,2018-01-11,0
3,16063,2018-01-12,0
4,16063,2019-01-01,0


In [63]:
df_con_reg['pk_partition'] = pd.to_datetime(df_con_reg['pk_partition'])

In [65]:
df_con_reg_merged = pd.merge(
    left = df_merged, 
    right = df_con_reg,
    on = ('pk_cid', 'pk_partition'),
    how = 'inner')

In [67]:
df_con_reg_merged.head(50)

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,cid,month_sale,product_ID,net_margin,reg_pension_plan
0,1050623,2018-01-01,KHE,1,03 - UNIVERSITARIO,49,H,23,113538.81,1050623,2018-02-01,3819,54.4,0
1,1050602,2018-01-01,KHE,1,03 - UNIVERSITARIO,37,V,23,91282.71,1050602,2019-04-01,3819,65.1,0
2,1050580,2018-01-01,KHE,1,03 - UNIVERSITARIO,27,H,23,77320.92,1050580,2018-10-01,3819,53.5,0
3,1050676,2018-01-01,KFC,0,02 - PARTICULARES,45,V,31,90431.85,1050676,2019-02-01,3819,60.4,0
4,1050660,2018-01-01,KFC,1,03 - UNIVERSITARIO,41,H,32,94683.27,1050660,2018-05-01,2234,74.2,0
5,1050660,2018-01-01,KFC,1,03 - UNIVERSITARIO,41,H,32,94683.27,1050660,2018-05-01,1364,5339.1,0
6,1050647,2018-01-01,KAT,1,02 - PARTICULARES,3,H,38,28359.36,1050647,2019-04-01,2234,88.2,0
7,1050647,2018-01-01,KAT,1,02 - PARTICULARES,3,H,38,28359.36,1050647,2019-04-01,1364,2426.7,0
8,1050647,2018-01-01,KAT,1,02 - PARTICULARES,3,H,38,28359.36,1050647,2018-07-01,3819,63.8,0
9,1050651,2018-01-01,KFC,1,02 - PARTICULARES,7,H,30,139502.28,1050651,2018-02-01,2234,58.5,0


In [81]:
df_con_reg_merged.loc[(df_con_reg_merged['reg_pension_plan'] == 1)]

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,cid,month_sale,product_ID,net_margin,reg_pension_plan
13,1050659,2018-01-01,KFC,1,02 - PARTICULARES,7,V,39,-1.00,1050659,2019-01-01,2673,1087.2,1
17,1050565,2018-01-01,KHE,1,03 - UNIVERSITARIO,46,H,33,54141.03,1050565,2018-06-01,4657,62.3,1
19,1050537,2018-01-01,KHE,1,03 - UNIVERSITARIO,50,H,23,214309.14,1050537,2018-05-01,3819,50.5,1
50,1050920,2018-01-01,KHE,1,03 - UNIVERSITARIO,46,H,25,-1.00,1050920,2018-12-01,4657,79.6,1
80,1050155,2018-01-01,KHE,1,03 - UNIVERSITARIO,45,V,27,54971.22,1050155,2018-03-01,3819,53.7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27622,1036039,2019-01-01,KHE,1,03 - UNIVERSITARIO,47,H,27,-1.00,1036039,2018-03-01,3819,61.2,1
27706,1036404,2019-01-01,KHE,1,03 - UNIVERSITARIO,47,H,23,91194.93,1036404,2019-01-01,2234,79.0,1
27707,1036404,2019-01-01,KHE,1,03 - UNIVERSITARIO,47,H,23,91194.93,1036404,2019-01-01,1364,9451.2,1
27708,1036404,2019-01-01,KHE,1,03 - UNIVERSITARIO,47,H,23,91194.93,1036404,2018-10-01,3819,60.8,1


In [72]:
df_con_reg_merged.loc[(df_con_reg_merged['pk_cid'] == 1050659)]

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,cid,month_sale,product_ID,net_margin,reg_pension_plan
13,1050659,2018-01-01,KFC,1,02 - PARTICULARES,7,V,39,-1.0,1050659,2019-01-01,2673,1087.2,1
19128,1050659,2019-01-01,KFC,1,02 - PARTICULARES,7,V,40,-1.0,1050659,2019-01-01,2673,1087.2,0


In [73]:
df_sales.loc[(df_sales['cid'] == 1050659)]

,cid,month_sale,product_ID,net_margin
5496,1050659,2019-01-01,2673,1087.2


In [79]:
df_cp = pd.read_csv('/Users/alinaoganesyan/Desktop/Master Data Science/Entregables/Proyecto final/git-proyecto/data/customer_products.csv')

In [80]:
df_cp.loc[(df_cp['pk_cid'] == 1050659)]

,Unnamed: 0,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
72,72,1050659,2018-01,0,0,0,0,0,0,0,1,1.0,1.0,1,1,1,0,0
472375,1243167,1050659,2018-02,0,0,0,0,0,0,0,1,1.0,1.0,1,1,0,0,0
693286,1847937,1050659,2018-03,0,0,0,0,0,0,0,1,1.0,1.0,1,1,1,0,0
969542,2507217,1050659,2018-04,0,0,0,0,0,0,0,1,1.0,1.0,1,1,0,0,0
981423,2519135,1050659,2018-05,0,0,0,0,0,0,0,1,1.0,1.0,1,1,0,0,0
1257113,3176867,1050659,2018-06,0,0,0,0,0,0,0,1,1.0,1.0,1,1,1,0,0
1581760,4371943,1050659,2018-07,0,0,0,0,0,0,0,1,1.0,1.0,1,1,0,0,0
2050067,5136523,1050659,2018-08,0,0,0,0,0,0,0,1,1.0,1.0,1,1,0,0,0
2331165,6053948,1050659,2018-09,0,0,0,0,0,0,0,1,1.0,1.0,1,1,0,0,0
2673578,6492072,1050659,2018-10,0,0,0,0,0,0,0,1,1.0,1.0,1,1,1,0,0


1. dejar solo un id de cliente
2. dejar solo una fecha de particion  
4. analizar si region_code debería ser un número por tener relación el código de provincia con algo cuantificable o si deberíamos hacer OHE (https://www.ine.es/daco/daco42/codmun/cod_ccaa_provincia.htm)
5. hacer OHE de entry_channel, segment, gender, region_code(?)
6. decidir qué producto vamos a querer predecir, quizás el que sea el más rentable? utilizar net_margin y si no hace falta, eliminar
7. empezar a construir el df final según las indicaciones de Mati
